In [6]:
import glob
import re
import pandas as pd
from yattag import Doc
import unicodedata

In [7]:
doc, tag, text = Doc().tagtext()
txt_file = glob.glob('dodf/*.txt')

all_contracts = []

instances = re.findall(r'[\s\S]+', 
            unicodedata.normalize('NFKD', open(txt_file[0], encoding='utf-8').read()))

for instance in instances:
    instance = instance.split('<END_OF_BLOCK>\nSECAO III\n')
    all_contracts.append(instance[-1])

secao_3 = all_contracts[0].split('\n')


### DFA

- l = lista com toda a seção 3 de um único dodf 
- i = posição atual da lista l -> inicia-se com 0
- l_acts = lista vazia que será preenchida com os atos
- j = posição atual da lista l_acts -> inicia-se com 0
- regex = regex do ato que está sendo procurado

In [67]:
def q0(l, i, l_acts, j, regex):
    """Estado que procura por um aviso de abertura"""

    if i == len(l):
        return

    if re.match(regex, l[i]):
        l_acts.append(l[i])
        q1(l, i + 1, l_acts, j, regex)

    else:
        q0(l, i + 1, l_acts, j, regex)


def q1(l, i, l_acts, j, regex):
    """Estado que procura a continuação do aviso de abertura"""

    if i == len(l):
        return

    regex_s = r'AVISOS?|EXTRATOS?|RESULTADOS?'

    if not re.match(regex, l[i]) and re.match(regex_s, l[i]) and '<END_OF_BLOCK>' in l[i+1]:
        q0(l, i + 1, l_acts, j + 1, regex)

    # if 'RESULTADOS DE LICITACAO' in l[i] or 'AVISO DE REABERTURA' in l[i]:
    #     q0(l, i + 1, l_acts, j + 1, regex)

    elif re.match(regex, l[i]):
        q0(l, i, l_acts, j + 1, regex)

    else:
        l_acts[j] += '\n' + l[i]
        q1(l, i + 1, l_acts, j, regex)



In [68]:
regex = r'AVISO\s+D[EO]\s+ABERTURA\s+D[EO]\s+LICITACAO|AVISO\s+D[EO]\s+ABERTURA'
l_acts = []
q0(secao_3, 0, l_acts, 0, regex)

In [69]:
l_acts

['AVISO DE ABERTURA\n<END_OF_BLOCK>\nPREGAO ELETRONICO SRP No 01/2021 - (UASG 450432)\n<END_OF_BLOCK>\nObjeto: Aquisicao de generos alimenticios pereciveis - Cortes Congelados de Frango  tipo\ncoxa e sobrecoxa sem pele, tipo file de peito de frango congelado sem pele e sem osso ou\npeito de frango congelado sem pele e sem osso ou ainda, peito de frango congelado\ndesossado sem pele, para o Programa de Alimentacao Escolar do Distrito Federal (PAE-DF),\nconforme especificacoes e condicoes estabelecidas no Termo de Referencia e seus Anexos.\nValor total estimado: R$ 45.261.284,29 (quarenta e cinco milhoes, duzentos e sessenta e um\nmil duzentos e oitenta e quatro reais e vinte e nove centavos). Cadastro das Propostas: a partir\nde 06/01/2021. Abertura das Propostas: 18/01/2021, as 10h, horario de Brasilia. O Edital\nestara disponivel nos enderecos eletronicos: www.comprasgovernamentais.gov.br e\nhttp://www.se.df.gov.br/pregao-eletronico-sistema-de-registro-de-precos/.\n<END_OF_BLOCK>\nBra

In [70]:
l_acts[0]

'AVISO DE ABERTURA\n<END_OF_BLOCK>\nPREGAO ELETRONICO SRP No 01/2021 - (UASG 450432)\n<END_OF_BLOCK>\nObjeto: Aquisicao de generos alimenticios pereciveis - Cortes Congelados de Frango  tipo\ncoxa e sobrecoxa sem pele, tipo file de peito de frango congelado sem pele e sem osso ou\npeito de frango congelado sem pele e sem osso ou ainda, peito de frango congelado\ndesossado sem pele, para o Programa de Alimentacao Escolar do Distrito Federal (PAE-DF),\nconforme especificacoes e condicoes estabelecidas no Termo de Referencia e seus Anexos.\nValor total estimado: R$ 45.261.284,29 (quarenta e cinco milhoes, duzentos e sessenta e um\nmil duzentos e oitenta e quatro reais e vinte e nove centavos). Cadastro das Propostas: a partir\nde 06/01/2021. Abertura das Propostas: 18/01/2021, as 10h, horario de Brasilia. O Edital\nestara disponivel nos enderecos eletronicos: www.comprasgovernamentais.gov.br e\nhttp://www.se.df.gov.br/pregao-eletronico-sistema-de-registro-de-precos/.\n<END_OF_BLOCK>\nBras

In [71]:
l_acts[1]

'AVISO DE ABERTURA DE LICITACAO\n<END_OF_BLOCK>\nPREGAO ELETRONICO No 55/2020\n<END_OF_BLOCK>\nProcesso: 00050-00011038/2020-83. TIPO: Menor Preco. MODO DE DISPUTA: Aberto.\nOBJETO: Aquisicao de bens de imagem, iluminacao e sonorizacao, visando atender\ndemanda da Secretaria de Estado de Seguranca Publica do Distrito Federal. VALOR\nESTIMADO: Sigiloso. PRAZOS: De entrega: 30 (trinta) dias corridos contados do\nrecebimento da nota de empenho ou da assinatura do contrato. Do Contrato: 90 (noventa)\ndias a partir da data da assinatura. DATA DA SESSAO PUBLICA: 18/01/2021 as 09:00\nhoras no www.gov.br/compras. UASG 450107. Edital tambem esta disponivel no\nhttp://www.ssp.df.gov.br/licitacoes/.\n<END_OF_BLOCK>\nBrasilia/DF, 05 de janeiro de 2021\n<END_OF_BLOCK>\nFERNANDO CESAR NEVES\n<END_OF_BLOCK>\nCoordenador, Substituto\n<END_OF_BLOCK>\nPOLICIA MILITAR DO DISTRITO FEDERAL\n<END_OF_BLOCK>\nDEPARTAMENTO DE LOGISTICA E FINANCAS\n<END_OF_BLOCK>\n \n<END_OF_BLOCK>'